In [ ]:
import os
import napari
import numpy as np
import pandas as pd
from tifffile import imread
from natsort import natsorted
import logging
import dask.array as da
from dask import delayed

# Display the tracks files in Napari
This should be run locally with accessible files like Spot masks and Track csv file (created with script `tracking.ipynb` on server).
You can then view the tracks in Napari and possibly also overlay the raw images.
Make sure you have proper packages, because the container does not support their running. 

In [1]:
# 1. Path to the directory containing your individual TIFF mask files.
MASK_DIRECTORY = "/home/mira/ibz_server/mvolosko/image_project/SDC1/1268_fast_imaging_01/spots/spot_segmentation/"

# 2. Path to the tracks CSV file generated by the tracking script.
TRACKS_CSV_PATH = "/home/mira/ibz_server/mvolosko/image_project/SDC1/1268_fast_imaging_01/tracks/napari_tracks_robust.csv"


In [ ]:
# --- Configuration ---
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logging.getLogger('tifffile').setLevel(logging.ERROR)

def robust_imread(filename):
    """
    Reads an image and squeezes out singleton dimensions to ensure consistency.
    For example, an image with shape (1, Z, Y, X) becomes (Z, Y, X).
    """
    return np.squeeze(imread(filename))

def load_masks_as_virtual_stack(directory: str):
    """
    Creates a Dask array, which represents the entire stack WITHOUT loading it all into memory.
    This version is robust to inconsistencies in image dimensions.
    """
    logging.info(f"Searching for TIFF files in: {directory}")
    files = natsorted([os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.tif', '.tiff'))])
    
    if not files:
        raise FileNotFoundError(f"No TIFF files were found in the directory: {directory}")

    # Get the shape and dtype from the first image using the ROBUST reader.
    sample = robust_imread(files[0])
    
    # Use dask.delayed to wrap our robust reader.
    lazy_reader = delayed(robust_imread)
    lazy_arrays = [lazy_reader(f) for f in files]

    # Create a stack of Dask arrays, ensuring each has the correct shape and dtype
    dask_arrays = [da.from_delayed(arr, shape=sample.shape, dtype=sample.dtype) for arr in lazy_arrays]
    
    # Stack them along the time axis (axis=0)
    stack = da.stack(dask_arrays, axis=0)
    stack = stack.compute_chunk_sizes()
    
    logging.info(f"Successfully created a 'virtual' stack with shape: {stack.shape}")
    logging.info(f"Memory usage will be minimal until you start scrolling in Napari.")
    return stack


def load_all_tracks(csv_path: str) -> np.ndarray:
    """
    Loads all tracking data from a CSV file. No filtering is needed for the virtual stack.
    """
    logging.info(f"Loading all tracks from: {csv_path}")
    df = pd.read_csv(csv_path)
    required_cols = ['track_id', 'time', 'z', 'y', 'x']
    return df[required_cols].to_numpy()


if __name__ == "__main__":
    if not os.path.isdir(MASK_DIRECTORY):
        logging.error(f"Mask directory not found: {MASK_DIRECTORY}")
    elif not os.path.isfile(TRACKS_CSV_PATH):
        logging.error(f"Tracks CSV not found: {TRACKS_CSV_PATH}")
    else:
        try:
            # 1. Create the VIRTUAL stack of all masks
            virtual_stack = load_masks_as_virtual_stack(MASK_DIRECTORY)

            # 2. Load ALL tracking data (it's small)
            tracks_data = load_all_tracks(TRACKS_CSV_PATH)

            # 3. Launch Napari
            logging.info("Launching Napari viewer...")
            viewer = napari.Viewer(ndisplay=3)
            
            # Napari natively understands Dask arrays and handles them efficiently
            viewer.add_labels(virtual_stack, name='Segmentation (Virtual Stack)')
            
            properties = {'track_id': tracks_data[:, 0]}
            tracks_layer = viewer.add_tracks(
                tracks_data,
                properties=properties,
                name='Object Tracks',
                tail_length=30
            )
            tracks_layer.color_by = 'track_id'
            
            napari.run()

        except (FileNotFoundError, ValueError) as e:
            logging.error(f"Could not start Napari due to an error: {e}")

INFO: Searching for TIFF files in: /home/mira/ibz_server/mvolosko/image_project/SDC1/1268_fast_imaging_01/spots/spot_segmentation/
INFO: Successfully created a 'virtual' stack with shape: (20, 27, 2560, 2560)
INFO: Memory usage will be minimal until you start scrolling in Napari.
INFO: Loading all tracks from: /home/mira/ibz_server/mvolosko/image_project/SDC1/1268_fast_imaging_01/tracks/napari_tracks_robust.csv
INFO: Launching Napari viewer...
INFO: No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


# Code to create animation from tracks in Napari

In [ ]:
from napari_animation import Animation
import logging

# --- Animation Configuration ---

# Set the desired frames per second for the output video
FPS = 20

# Set how long to pause on each data frame (in video frames).
# Since FPS is 20, a pause of 20 equals a 1-second hold.
PAUSE_DURATION_FRAMES = 20

# Set how many video frames the initial zoom-in should take.
ZOOM_DURATION_FRAMES = 25


# --- Animation Script ---

# 1. Create an Animation object from the existing viewer
animation = Animation(viewer)
total_data_frames = virtual_stack.shape[0]

# --- Define the animation's key states ---

# Keyframe 1: The very beginning, unzoomed.
viewer.reset_view()
viewer.dims.set_current_step(0, 0)
animation.capture_keyframe() # This is the first frame of the animation

# Keyframe 2: The zoomed-in state.
# The animation creates a smooth zoom transition to this state.
viewer.camera.zoom *= 3
animation.capture_keyframe(steps=ZOOM_DURATION_FRAMES)

# 3. Loop through each data frame to create the "stop-motion" effect
logging.info(f"Creating stop-motion sequence for {total_data_frames} frames...")
for i in range(total_data_frames):
    # Move the viewer to the next data frame
    viewer.dims.set_current_step(0, i)

    # Capture this view and hold it for the specified pause duration
    animation.capture_keyframe(steps=PAUSE_DURATION_FRAMES)

# --- Generate the video ---
logging.info("Generating animation... this may take a moment.")
animation.animate(
    "1268_01.mp4",
    canvas_only=True,
    fps=FPS
)

logging.info("Animation saved to '1268_01.mp4'")

INFO: Creating stop-motion sequence for 20 frames...
INFO: Generating animation... this may take a moment.


Rendering frames...


100%|██████████| 426/426 [15:15<00:00,  2.15s/it]
INFO: Animation saved to '1268_01.mp4'


# Code for plotting 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file into a pandas DataFrame
# Make sure to replace csv with the actual name of your file
try:
    df = pd.read_csv('/home/mira/ibz_server/mvolosko/image_project/SDC1/1268_fast_imaging_01/tracks/analysis_results_robust.csv')
except FileNotFoundError:
    print("Error: The csv file  was not found.")
    print("Please make sure the file is in the same directory as the script,")
    print("or provide the full file path.")
    exit()


# Filter out the rows where time is 0
df_filtered = df[df['time'] != 0]
events_to_exclude = ['bridged', 'bridged_disappearance', 'disappearance']
df_filtered = df_filtered[~df_filtered['event_type'].isin(events_to_exclude)]
df_filtered['event_type'] = df_filtered['event_type'].replace('terminal', 'terminated')

# Group by time and event_type, and count the occurrences
event_counts = df_filtered.groupby(['time', 'event_type']).size().reset_index(name='count')

# Pivot the table to have event_types as columns
pivot_df = event_counts.pivot(index='time', columns='event_type', values='count').fillna(0)


# --- Plotting and Label Conversion ---
plt.style.use('seaborn-v0_8-muted')
fig, ax = plt.subplots(figsize=(14, 7))

pivot_df.plot(kind='line', ax=ax, marker='', linestyle='-', linewidth=4)

# Setting the title and labels
ax.set_title('Event Types Over Time', fontsize=16)
ax.set_xlabel('Time (min)', fontsize=16)
ax.set_ylabel('Number of Events', fontsize=16)
ax.legend(title='Event Type')
ax.grid(True)

# NEW: Convert x-axis from frame number to real time
minutes_per_frame = 30.0 / 20.0

# Get the tick locations (which are frame numbers)
tick_locations = ax.get_xticks()

# Create the new labels in MM:SS format
new_labels = []
for frame in tick_locations:
    # Ignore ticks outside the data range (e.g., < 0)
    if frame >= 0:
        total_minutes = frame * minutes_per_frame
        minutes = int(total_minutes)
        new_labels.append(f'{minutes:02d}')

# Set the new discrete labels on the plot
ax.set_xticks(tick_locations) # Ensures ticks are set at integer frame values
ax.set_xticklabels(new_labels)


# Final adjustments
plt.tight_layout() # Adjusts plot to prevent labels from overlapping
plt.savefig('event_types_real_time.png')

print("Plot saved as event_types_real_time.png")


In [ ]:
# --- Calculate number of tracks per timepoint (replicates = tracks) ---
track_counts = pivot_df.apply(lambda row: (row > 0).astype(int), axis=1)

# For each timepoint: mean number of tracks and SEM across track replicates
track_dynamics = track_counts.sum(axis=1).groupby(level=0).agg(['mean', 'sem'])

# --- Plotting ---
plt.style.use('seaborn-v0_8-muted')
fig, ax = plt.subplots(figsize=(14, 7))

# Mean + SEM (like your size_dynamics)
ax.errorbar(
    track_dynamics.index,
    track_dynamics['mean'],
    yerr=track_dynamics['sem'],
    capsize=4,
    marker='o',
    linestyle='-',
    linewidth=2
)

# Titles and labels
ax.set_title("Tracks per Timepoint", fontsize=16)
ax.set_xlabel("Time (mins)", fontsize=12)
ax.set_ylabel("Number of Tracks", fontsize=12)
ax.grid(True)

# --- Convert x-axis from frames to minutes ---
minutes_per_frame = 30.0 / 20.0  # adjust to your acquisition rate

tick_locations = ax.get_xticks()
new_labels = []
for frame in tick_locations:
    if frame >= 0:
        total_minutes = frame * minutes_per_frame
        minutes = int(total_minutes)
        new_labels.append(f'{minutes:02d}')

ax.set_xticks(tick_locations)
ax.set_xticklabels(new_labels)

plt.tight_layout()
plt.savefig('tracks_vs_time.png')
print("Plot saved as tracks_vs_time.png")


In [ ]:

# --- NEW: Calculate Size Dynamics ---
# Group by time and calculate the mean and standard error of the mean (sem) for the 'area'
size_dynamics = df_filtered.groupby('time')['area'].agg(['mean', 'sem'])


# --- Plotting and Label Conversion ---
fig, ax = plt.subplots(figsize=(14, 7))

# Plot the mean area with the SEM as error bars (yerr)
size_dynamics['mean'].plot(ax=ax, yerr=size_dynamics['sem'], capsize=4, marker='o', linestyle='-')


# Setting the title and labels
ax.set_title('Size Dynamics: Mean Area vs. Time (±SEM)', fontsize=16)
ax.set_xlabel('Time (mins)', fontsize=12)
ax.set_ylabel('Mean Area', fontsize=12)
ax.grid(True)

# Convert x-axis from frame number to real time
minutes_per_frame = 30.0 / 20.0

# Get the tick locations (which are frame numbers)
tick_locations = ax.get_xticks()

# Create the new labels in MM:SS format
new_labels = []
for frame in tick_locations:
    # Ignore ticks outside the data range (e.g., < 0)
    if frame >= 0:
        total_minutes = frame * minutes_per_frame
        minutes = int(total_minutes)
        new_labels.append(f'{minutes:02d}')

# Set the new discrete labels on the plot
ax.set_xticks(tick_locations) # Ensures ticks are set at integer frame values
ax.set_xticklabels(new_labels)


# Final adjustments
plt.tight_layout()
plt.savefig('mean_area_vs_time.png')

print("Plot saved as mean_area_vs_time.png")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# --- 1. Load and Prepare Data 
try:
    df = pd.read_csv('/home/mira/ibz_server/mvolosko/image_project/SDC1/1268_fast_imaging_01/tracks/analysis_results_robust.csv')
except FileNotFoundError:
    print("Error: The file 'your_file.csv' was not found.")
    print("Please make sure the file is in the same directory as the script,")
    print("or provide the full file path.")
    exit()


# NEW: Handle duplicates for a given track_id at the same time point
df.drop_duplicates(subset=['track_id', 'time'], keep='last', inplace=True)

events_to_exclude = ['bridged', 'bridged_disappearance', 'disappearance']
df = df[~df['event_type'].isin(events_to_exclude)]
df['event_type'] = df['event_type'].replace('terminal', 'terminated')

# --- 2. Create the "Long-Form" Lifecycle Data ---
print("Building track lifecycles...")
# Find the start and end time for each track
track_lifetimes = df.groupby('track_id')['time'].agg(['min', 'max']).reset_index()

lifecycle_data = []
# For each track, create a record for every frame it exists
for _, row in track_lifetimes.iterrows():
    track_id = row['track_id']
    for time in range(row['min'], row['max'] + 1):
        # Default state is 'persisting'
        lifecycle_data.append({'track_id': track_id, 'time': time, 'event_type': 'persisting'})

# Convert the list of dictionaries to a DataFrame
lifecycle_df = pd.DataFrame(lifecycle_data)

# Now, update the 'persisting' state with actual events from the original data
# We set indices to easily align and update the data
lifecycle_df.set_index(['track_id', 'time'], inplace=True)
original_events = df.set_index(['track_id', 'time'])
lifecycle_df.update(original_events)
lifecycle_df.reset_index(inplace=True)


# --- 3. Create the Plotting Matrix ---
print("Creating plotting matrix...")
# Map event names to numbers for plotting
event_names = ['persisting', 'appearance', 'split', 'merge', 'terminated']
event_ids = {name: i for i, name in enumerate(event_names)}
lifecycle_df['event_id'] = lifecycle_df['event_type'].map(event_ids).fillna(-1) # Use -1 for missing

# Pivot to create the matrix: rows=tracks, columns=time, values=event_id
plot_matrix = lifecycle_df.pivot(index='track_id', columns='time', values='event_id')


# --- 4. Generate the Plot ---
print("Generating plot...")
# Define a custom color for each event type
colors = ['#d3d3d3', '#2ca02c', '#ff7f0e', '#d62728', '#1f77b4'] # gray, green, orange, purple, red, blue, brown
cmap = mcolors.ListedColormap(colors)
bounds = np.arange(len(event_names) + 1) - 0.5
norm = mcolors.BoundaryNorm(bounds, cmap.N)

fig, ax = plt.subplots(figsize=(16, 25))
im = ax.imshow(plot_matrix, aspect='auto', cmap=cmap, norm=norm, interpolation='none')

# Create a colorbar legend
cbar = fig.colorbar(im, ax=ax, ticks=np.arange(len(event_names)))
cbar.set_ticklabels(event_names)
cbar.set_label('Event Type', rotation=270, labelpad=15)

# Formatting the plot
ax.set_title('Track Lifecycles', fontsize=18)
ax.set_xlabel('Time (frames)', fontsize=12)
ax.set_ylabel('Track ID', fontsize=12)
# ax.set_xlim(0.5, 10.5)
ax.set_ylim(0, 300)
# Save the plot
plt.tight_layout()
plt.savefig('track_lifecycles.png')

print("\nPlot saved as track_lifecycles.png")